# relic data, excel file merging notebook 

- 2024.12, tk

In [1]:
import time, os
from datetime import datetime
import pandas as pd
from collections import deque
from openpyxl import Workbook, load_workbook
from openpyxl.worksheet.worksheet import Worksheet
from IPython.display import display
from pprint import pprint

In [2]:
start_time = time.time()

## Enviroments

In [3]:
# file path - please update
ifile1 = './data/전통문양 메타데이터 통합관리_v2.1.xlsx'
ifile2 = './data/캡션검수_1125.xlsx'
ofile1 = './data/master (20.전통문양).xlsx'

## Functions

In [4]:
def filename_suffix()->str:
    return f'{os.getpid()}-{datetime.now().strftime("%H%M%S")}'

In [5]:
def nonempty_rows_count(sheet:Worksheet) -> int: 
    non_empty_rows = 0
    for row in sheet.iter_rows():
        if any(cell.value for cell in row):
            non_empty_rows += 1
    return non_empty_rows


In [6]:
def workbook_info(workbook: Workbook, head:int=5, tail:int=5, datadisplay:bool=False) -> None:
    """print workbook and its sheet information"""
    
    # Get and print basic workbook properties
    print("\nWorkbook Properties:")
    #print(f"Title: {workbook.properties.title}")
    #print(f"Subject: {workbook.properties.subject}")
    #print(f"Created: {workbook.properties.created}")
    print(f"Modified: {workbook.properties.modified}")
    print(f"Number of sheets: {len(workbook.sheetnames)}")
    print(f"Names of sheets: {workbook.sheetnames}")
        
    # Get sheet properties
    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
        print(f"\nSheet: {sheet.title}")
        #print(f"  Dimensions: {sheet.dimensions}")  # Example: 'A1:C10'
        print(f"  Columns: {sheet.max_column}")
        print(f"  Rows: {sheet.max_row}")
        print(f"  Nonempty rows: {nonempty_rows_count(sheet)}")


        if datadisplay:
            # Print a sample of the first 5 rows to understand content
            if head:
                print(f"  First {head} rows:")
                for row in sheet.iter_rows(min_row=1, max_row=head, values_only=True):
                    print(f"    {row}")        
            if tail:
                print(f"  Tail {head} rows:")
                for row in sheet.iter_rows(min_row=sheet.max_row-tail, values_only=True):
                    print(f"    {row}")                        
        

In [7]:
def truncate_rows_except_headers(sheet: Worksheet, header_row_count=1)->None:
    # Find the number of rows in the sheet
    total_rows = sheet.max_row
    
    # Loop through rows below the header and delete them
    # for row_idx in range(total_rows, header_row_count, -1):  # Start from the bottom to avoid indexing issues
    #     sheet.delete_rows(row_idx)
    sheet.delete_rows(header_row_count+1, total_rows-header_row_count) 
    
    print(f'sheet={sheet.title} truncate rows={total_rows-header_row_count} except header_row_count={header_row_count}')

In [8]:
def text_and_width(sheet:Worksheet, max_width:int=28)->None:
    # set as text and width min(width, maxwidth) 

    # Iterate over each column and set the text type and width
    for column in sheet.columns:
        column_letter = column[0].column_letter
        sheet.column_dimensions[column_letter].number_format = '@'  # set text type 
        
        max_length = max(len(str(cell.value)) for cell in column)
        sheet.column_dimensions[column_letter].width = min(max_length + 2, max_width)
                

In [9]:
def dataframe_info(df:pd.DataFrame, head:int=5, tail:int=5, datadisplay:bool=False) -> None:
    print(f'shape: {df.shape}')
    print(f'columns:\n{df.columns.tolist()}')
    
    if datadisplay:
        if head:
            print(f'head {head} rows:')
            display(df.head(head))
        if tail:
            print(f'tail {tail} rows:')
            display(df.tail(tail))

## Excel file information

In [10]:
# meta  file 
iwb1 = load_workbook(ifile1)
#workbook_info(iwb1, head=3, tail=3, datadisplay=True)
workbook_info(iwb1, datadisplay=False)
iwb1 = None # release memory


Workbook Properties:
Modified: 2024-12-13 07:29:04.918547
Number of sheets: 11
Names of sheets: ['진행상황', '10.중앙박물관(HCI+)', '12.국가유산진흥원 발굴유물DB', '11.국가유산진흥원 직접촬영지원', '13.중앙박물관(LiST 1차)', '15.문화정보원(1차)', '14.중앙박물관(LiST 2차)', '17.문화정보원(2차)', '16.전통문화포털 2D문양', '원천유물명 검토', '시트11']

Sheet: 진행상황
  Columns: 23
  Rows: 987
  Nonempty rows: 35

Sheet: 10.중앙박물관(HCI+)
  Columns: 34
  Rows: 6292
  Nonempty rows: 6292

Sheet: 12.국가유산진흥원 발굴유물DB
  Columns: 32
  Rows: 448
  Nonempty rows: 448

Sheet: 11.국가유산진흥원 직접촬영지원
  Columns: 39
  Rows: 2239
  Nonempty rows: 2239

Sheet: 13.중앙박물관(LiST 1차)
  Columns: 32
  Rows: 1590
  Nonempty rows: 1590

Sheet: 15.문화정보원(1차)
  Columns: 36
  Rows: 2687
  Nonempty rows: 2687

Sheet: 14.중앙박물관(LiST 2차)
  Columns: 36
  Rows: 13387
  Nonempty rows: 13387

Sheet: 17.문화정보원(2차)
  Columns: 35
  Rows: 6960
  Nonempty rows: 6960

Sheet: 16.전통문화포털 2D문양
  Columns: 29
  Rows: 999
  Nonempty rows: 583

Sheet: 원천유물명 검토
  Columns: 12
  Rows: 28950
  Nonempty rows: 28950

Sheet: 시트11


In [11]:
# caption file 
iwb2 = load_workbook(ifile2)
workbook_info(iwb2, datadisplay=False)
iwb2 = None # release memory


Workbook Properties:
Modified: 2024-12-13 07:29:26.223184
Number of sheets: 14
Names of sheets: ['캡션검수 1125', '캡션검수1126(수정)', '캡션검수1201', '캡션검수1202', '캡션검수1205', '캡션검수1206', '캡션검수1207(1,4문단)', '캡션검수1209(1,4문단)', '캡션검수1210(1,4문단)', '캡션검수1211(1,4문단)', '캡션검수1212(1,4문단)', '캡션검수1213(1문단,4문단)', '캡션검수1213-2', '한글,한자 포함된 캡션']

Sheet: 캡션검수 1125
  Columns: 34
  Rows: 1898
  Nonempty rows: 1898

Sheet: 캡션검수1126(수정)
  Columns: 24
  Rows: 3235
  Nonempty rows: 3235

Sheet: 캡션검수1201
  Columns: 30
  Rows: 1286
  Nonempty rows: 1286

Sheet: 캡션검수1202
  Columns: 23
  Rows: 1697
  Nonempty rows: 1697

Sheet: 캡션검수1205
  Columns: 30
  Rows: 1167
  Nonempty rows: 1167

Sheet: 캡션검수1206
  Columns: 30
  Rows: 2494
  Nonempty rows: 2494

Sheet: 캡션검수1207(1,4문단)
  Columns: 30
  Rows: 1542
  Nonempty rows: 1542

Sheet: 캡션검수1209(1,4문단)
  Columns: 30
  Rows: 1529
  Nonempty rows: 1529

Sheet: 캡션검수1210(1,4문단)
  Columns: 30
  Rows: 2561
  Nonempty rows: 2351

Sheet: 캡션검수1211(1,4문단)
  Columns: 30
  Rows: 1683
  Nonemp

In [12]:
# master file 
owb1 = load_workbook(ofile1)
workbook_info(owb1, datadisplay=False)
# owb1 used later


Workbook Properties:
Modified: 2024-12-13 07:29:36.679685
Number of sheets: 1
Names of sheets: ['master']

Sheet: master
  Columns: 38
  Rows: 27058
  Nonempty rows: 27058


## Excel file read to dataframe

In [13]:
# meta file

# sheets_all: ['진행상황', '10.중앙박물관(HCI+)', '12.국가유산진흥원 발굴유물DB', '11.국가유산진흥원 직접촬영지원', '13.중앙박물관(LiST 1차)', '15.문화정보원(1차)', '14.중앙박물관(LiST 2차)', '17.문화정보원(2차)', '16.전통문화포털 2D문양', '원천유물명 검토', '시트11']
# sheets_excluded =  ['진행상황', '16.전통문화포털 2D문양', '원천유물명 검토', '시트11']

metadfs = {}
columns = ['relic_id','relic_name','relic_common_name','pattern_usage','pattern_usage_detail','material','source','era','pattern_type','pattern_type_detail']  # 10

# check skiprows and usecols per sheet
metadfs['meta10'] = pd.read_excel(ifile1, sheet_name='10.중앙박물관(HCI+)',           skiprows=4, usecols=[0,2,4,5,6,7,8,9,10,12],  header=None, names=columns, na_filter=False, dtype='object', engine='openpyxl',)  # all column data type as string
metadfs['meta11'] = pd.read_excel(ifile1, sheet_name='11.국가유산진흥원 직접촬영지원', skiprows=5, usecols=[0,2,4,5,6,7,8,9,10,12],  header=None, names=columns, na_filter=False, dtype='object', engine='openpyxl',)
metadfs['meta12'] = pd.read_excel(ifile1, sheet_name='12.국가유산진흥원 발굴유물DB',   skiprows=5, usecols=[0,2,4,5,6,7,8,9,10,12],  header=None, names=columns, na_filter=False, dtype='object', engine='openpyxl',)
metadfs['meta13'] = pd.read_excel(ifile1, sheet_name='13.중앙박물관(LiST 1차)',       skiprows=5, usecols=[0,2,4,5,6,7,8,9,10,12],  header=None, names=columns, na_filter=False, dtype='object', engine='openpyxl',)
metadfs['meta14'] = pd.read_excel(ifile1, sheet_name='14.중앙박물관(LiST 2차)',       skiprows=5, usecols=[0,2,4,5,6,7,8,9,10,12],  header=None, names=columns, na_filter=False, dtype='object', engine='openpyxl',)    
metadfs['meta15'] = pd.read_excel(ifile1, sheet_name='15.문화정보원(1차)',            skiprows=4, usecols=[0,2,5,6,7,8,9,10,11,13], header=None, names=columns, na_filter=False, dtype='object', engine='openpyxl',)    
metadfs['meta17'] = pd.read_excel(ifile1, sheet_name='17.문화정보원(2차)',            skiprows=3, usecols=[0,2,4,5,6,7,8,9,10,12],  header=None, names=columns, na_filter=False, dtype='object', engine='openpyxl',)    

metadf = pd.concat([val for key,val in metadfs.items()])

meta_all_rows = metadf.shape[0]


In [14]:

# Strip whitespace from string columns
string_cols = metadf.select_dtypes(include='object').columns
metadf[string_cols] = metadf[string_cols].apply(lambda x: x.str.strip())

# remove invalid rows 
meta_invalid_df = metadf[(metadf['relic_id'].isnull()) | (metadf['relic_id'].str.len() < 1)]
metadf = metadf[(metadf['relic_id'].notnull()) & (metadf['relic_id'].str.len() > 0)]

# remove duplicate rows 
meta_duplicate_df = metadf[metadf.duplicated(subset=['relic_id'])]
metadf = metadf.drop_duplicates(subset=['relic_id']) 

meta_invalid_rows = meta_invalid_df.shape[0]
meta_duplicate_rows = meta_duplicate_df.shape[0]
meta_valid_rows = metadf.shape[0]

In [15]:
print(f'meta rows all={meta_all_rows}, valid={meta_valid_rows}, invalid={meta_invalid_rows}, duplicate={meta_duplicate_rows}')

meta rows all=33572, valid=33563, invalid=2, duplicate=7


In [16]:
dataframe_info(metadf, datadisplay=False)

shape: (33563, 10)
columns:
['relic_id', 'relic_name', 'relic_common_name', 'pattern_usage', 'pattern_usage_detail', 'material', 'source', 'era', 'pattern_type', 'pattern_type_detail']


In [17]:
dataframe_info(meta_invalid_df, datadisplay=False)

shape: (2, 10)
columns:
['relic_id', 'relic_name', 'relic_common_name', 'pattern_usage', 'pattern_usage_detail', 'material', 'source', 'era', 'pattern_type', 'pattern_type_detail']


In [18]:
dataframe_info(meta_invalid_df, datadisplay=False)

shape: (2, 10)
columns:
['relic_id', 'relic_name', 'relic_common_name', 'pattern_usage', 'pattern_usage_detail', 'material', 'source', 'era', 'pattern_type', 'pattern_type_detail']


In [19]:
# debug saving
metadf.to_excel(f'./data/metabook-{filename_suffix()}.xlsx', index=False)
if meta_invalid_rows:
    meta_invalid_df.to_excel(f'./data/metabook-invalid-{filename_suffix()}.xlsx', index=False)
if meta_duplicate_rows:    
    meta_duplicate_df.to_excel(f'./data/metabook-duplicate-{filename_suffix()}.xlsx', index=False)

In [20]:
# caption file 

# sheets_all =  ['캡션검수 1125', '캡션검수1126(수정)', '캡션검수1201', '캡션검수1202', '캡션검수1205', '캡션검수1206', '캡션검수1207(1,4문단)', '캡션검수1209(1,4문단)', '캡션검수1210(1,4문단)']
# sheets_exclude = []

capdfs = {}
columns = ['relic_id','1문단','2문단','3문단','4문단','5문단','6문단','1paragraph','2paragraph','3paragraph','4paragraph','5paragraph','6paragraph']  # 13

# check skiprows, usecols 
capdfs['cap1125']  = pd.read_excel(ifile2, sheet_name='캡션검수 1125',           skiprows=2, usecols=[1, 8,9,10,11,12,13,  18,19,20,21,22,23],  header=None, names=columns, na_filter=False, dtype='object', engine='openpyxl',)
capdfs['cap1126a'] = pd.read_excel(ifile2, sheet_name='캡션검수1126(수정)',      skiprows=1, usecols=[1, 8,9,10,11,12,13,  18,19,20,21,22,23],  header=None, names=columns, na_filter=False, dtype='object', engine='openpyxl',)

capdfs['cap1201']  = pd.read_excel(ifile2, sheet_name='캡션검수1201',            skiprows=1, usecols=[0, 7,8,9,10,11,12,  17,18,19,20,21,22],  header=None, names=columns, na_filter=False, dtype='object', engine='openpyxl',)
capdfs['cap1202']  = pd.read_excel(ifile2, sheet_name='캡션검수1202',            skiprows=1, usecols=[0, 7,8,9,10,11,12,  17,18,19,20,21,22],  header=None, names=columns, na_filter=False, dtype='object', engine='openpyxl',)
capdfs['cap1205']  = pd.read_excel(ifile2, sheet_name='캡션검수1205',            skiprows=1, usecols=[0, 7,8,9,10,11,12,  17,18,19,20,21,22],  header=None, names=columns, na_filter=False, dtype='object', engine='openpyxl',)
capdfs['cap1206']  = pd.read_excel(ifile2, sheet_name='캡션검수1206',            skiprows=1, usecols=[0, 7,8,9,10,11,12,  17,18,19,20,21,22],  header=None, names=columns, na_filter=False, dtype='object', engine='openpyxl',)
capdfs['cap1207']  = pd.read_excel(ifile2, sheet_name='캡션검수1207(1,4문단)',   skiprows=1, usecols=[0, 7,8,9,10,11,12,  17,18,19,20,21,22],  header=None, names=columns, na_filter=False, dtype='object', engine='openpyxl',)
capdfs['cap1209']  = pd.read_excel(ifile2, sheet_name='캡션검수1209(1,4문단)',   skiprows=1, usecols=[0, 7,8,9,10,11,12,  17,18,19,20,21,22],  header=None, names=columns, na_filter=False, dtype='object', engine='openpyxl',)
capdfs['cap1210']  = pd.read_excel(ifile2, sheet_name='캡션검수1210(1,4문단)',   skiprows=1, usecols=[0, 7,8,9,10,11,12,  17,18,19,20,21,22],  header=None, names=columns, na_filter=False, dtype='object', engine='openpyxl',)

capdf = pd.concat([val for key,val in capdfs.items()])

cap_all_rows = capdf.shape[0]


In [21]:
# Strip whitespace from string columns
string_cols = capdf.select_dtypes(include='object').columns
capdf[string_cols] = capdf[string_cols].apply(lambda x: x.str.strip())

# remove invalid rows 
cap_invalid_df = capdf[(capdf['relic_id'].isnull()) | (capdf['relic_id'].str.len() < 1)]
capdf = capdf[(capdf['relic_id'].notnull()) & (capdf['relic_id'].str.len() > 0)]

# remove duplicate rows 
cap_duplicate_df = capdf[capdf.duplicated(subset=['relic_id'])]
capdf = capdf.drop_duplicates(subset=['relic_id']) 

cap_invalid_rows = cap_invalid_df.shape[0]
cap_duplicate_rows = cap_duplicate_df.shape[0]
cap_valid_rows = capdf.shape[0]


In [22]:
print(f'caption rows all={cap_all_rows}, valid={cap_valid_rows}, invalid={cap_invalid_rows}, duplicate={cap_duplicate_rows}')

caption rows all=17189, valid=15920, invalid=0, duplicate=1269


In [23]:
dataframe_info(capdf, datadisplay=False)

shape: (15920, 13)
columns:
['relic_id', '1문단', '2문단', '3문단', '4문단', '5문단', '6문단', '1paragraph', '2paragraph', '3paragraph', '4paragraph', '5paragraph', '6paragraph']


In [24]:
dataframe_info(cap_invalid_df, datadisplay=False) 

shape: (0, 13)
columns:
['relic_id', '1문단', '2문단', '3문단', '4문단', '5문단', '6문단', '1paragraph', '2paragraph', '3paragraph', '4paragraph', '5paragraph', '6paragraph']


In [25]:
dataframe_info(cap_duplicate_df, datadisplay=False) 

shape: (1269, 13)
columns:
['relic_id', '1문단', '2문단', '3문단', '4문단', '5문단', '6문단', '1paragraph', '2paragraph', '3paragraph', '4paragraph', '5paragraph', '6paragraph']


In [26]:
# debug saving
capdf.to_excel(f'./data/capbook-{filename_suffix()}.xlsx', index=False)
if cap_invalid_rows:
    cap_invalid_df.to_excel(f'./data/capbook-invalid-{filename_suffix()}.xlsx', index=False)
if cap_duplicate_rows:    
    cap_duplicate_df.to_excel(f'./data/capbook-duplciate-{filename_suffix()}.xlsx', index=False)

In [27]:
# master file 

# sheets_all =  ['master']

# columns_all:
master_columns = [
    'check','relic_id_img','relic_id','relic_name','relic_name_eng','relic_common_name','relic_common_name_eng','pattern_usage','pattern_usage_id','pattern_usage_detail','material','source','era','pattern_type','pattern_type_id','pattern_type_detail','pattern_type_detail_eng',
    'relic_no','collect','photo_date','photo_eqipment','color','object_img_resolution','pattern_img_resolution','object_file_name','relic_symbol',
    '1문단','2문단','3문단','4문단','5문단','6문단','1paragraph','2paragraph','3paragraph','4paragraph','5paragraph','6paragraph'
]
master_keep_names = [
    'check','relic_id_img','relic_name_eng','relic_common_name_eng','pattern_usage_id','pattern_type_id','pattern_type_detail_eng','relic_no','collect','photo_date','photo_eqipment','color','object_img_resolution','pattern_img_resolution','object_file_name','relic_symbol'
]
master_keep_index = [master_columns.index(cn) for cn in master_keep_names] 
#print(master_keep_index)

master_heads = 4

# check skiprows, usecols 
masdf = pd.read_excel(ofile1, sheet_name='master',  header=None,  skiprows=master_heads, usecols=master_keep_index, names=master_keep_names, na_filter=False, dtype='object', engine='openpyxl',)

mas_all_rows = masdf.shape[0]


In [28]:
# Strip whitespace from string columns
string_cols = masdf.select_dtypes(include='object').columns
masdf[string_cols] = masdf[string_cols].apply(lambda x: x.str.strip())

# remove invalid rows 
mas_invalid_df = masdf[(masdf['relic_id_img'].isnull()) | (masdf['relic_id_img'].str.len() < 1)]
masdf = masdf[(masdf['relic_id_img'].notnull()) & (masdf['relic_id_img'].str.len() > 0)]

# remove duplicate rows 
mas_duplicate_df = masdf[masdf.duplicated(subset=['relic_id_img'])]
masdf = masdf.drop_duplicates(subset=['relic_id_img']) 

mas_invalid_rows = mas_invalid_df.shape[0]
mas_duplicate_rows = mas_duplicate_df.shape[0]
mas_valid_rows = masdf.shape[0]

In [29]:
print(f'master rows all={mas_all_rows}, valid={mas_valid_rows}, invalid={mas_invalid_rows}, duplicate={mas_duplicate_rows}')

master rows all=27054, valid=27054, invalid=0, duplicate=0


In [30]:
dataframe_info(masdf, datadisplay=False)

shape: (27054, 16)
columns:
['check', 'relic_id_img', 'relic_name_eng', 'relic_common_name_eng', 'pattern_usage_id', 'pattern_type_id', 'pattern_type_detail_eng', 'relic_no', 'collect', 'photo_date', 'photo_eqipment', 'color', 'object_img_resolution', 'pattern_img_resolution', 'object_file_name', 'relic_symbol']


In [31]:
dataframe_info(mas_invalid_df, datadisplay=False)

shape: (0, 16)
columns:
['check', 'relic_id_img', 'relic_name_eng', 'relic_common_name_eng', 'pattern_usage_id', 'pattern_type_id', 'pattern_type_detail_eng', 'relic_no', 'collect', 'photo_date', 'photo_eqipment', 'color', 'object_img_resolution', 'pattern_img_resolution', 'object_file_name', 'relic_symbol']


In [32]:
dataframe_info(mas_duplicate_df, datadisplay=False)

shape: (0, 16)
columns:
['check', 'relic_id_img', 'relic_name_eng', 'relic_common_name_eng', 'pattern_usage_id', 'pattern_type_id', 'pattern_type_detail_eng', 'relic_no', 'collect', 'photo_date', 'photo_eqipment', 'color', 'object_img_resolution', 'pattern_img_resolution', 'object_file_name', 'relic_symbol']


In [33]:
# debug saving
masdf.to_excel(f'./data/masbook-{filename_suffix()}.xlsx', index=False)
if mas_invalid_rows:
    mas_invalid_df.to_excel(f'./data/masbook-invalid-{filename_suffix()}.xlsx', index=False)
if mas_duplicate_rows:    
    mas_duplicate_df.to_excel(f'./data/masbook-duplicate-{filename_suffix()}.xlsx', index=False)

In [34]:
# join masdf, metadf, capdf 

# Find values in masdf['relic_id'] that not metadf['relic_id'] 
meta_notfound_values = masdf['relic_id_img'][~masdf['relic_id_img'].isin(metadf['relic_id'])]
meta_notfound_rows = meta_notfound_values.count()

# Find values in masdf['relic_id'] that not metadf['relic_id'] 
cap_notfound_values = masdf['relic_id_img'][~masdf['relic_id_img'].isin(capdf['relic_id'])]
cap_notfound_rows = cap_notfound_values.count()

# inner join
innerdf = masdf.merge(metadf, how='inner', left_on='relic_id_img', right_on='relic_id', suffixes=['_left1','_right1'], copy=True)  # relic_id added
innerdf = innerdf.merge(capdf, how='inner', left_on='relic_id_img', right_on='relic_id', suffixes=['_left2','_right2'], copy=True) # relic_id_left2, relic_id_right2  
mas_inner_rows = innerdf.shape[0]

# left outer join
outerdf = masdf.merge(metadf, how='left', left_on='relic_id_img', right_on='relic_id', suffixes=['_left1','_right1'], copy=True)  # relic_id added
outerdf = outerdf.merge(capdf, how='left', left_on='relic_id_img', right_on='relic_id', suffixes=['_left2','_right2'], copy=True) # relic_id_left2, relic_id_right2  
mas_outer_rows = outerdf.shape[0]

In [35]:
# delete relic_id_left2, relic_id_right2
print(innerdf.columns.tolist())
innerdf = innerdf.drop([ 'relic_id_left2','relic_id_right2'], axis=1)
# assign relic_id with relic_id_img 
innerdf['relic_id'] = innerdf['relic_id_img']
# reorder columns
innerdf = innerdf[master_columns]
print(innerdf.columns.tolist())

['check', 'relic_id_img', 'relic_name_eng', 'relic_common_name_eng', 'pattern_usage_id', 'pattern_type_id', 'pattern_type_detail_eng', 'relic_no', 'collect', 'photo_date', 'photo_eqipment', 'color', 'object_img_resolution', 'pattern_img_resolution', 'object_file_name', 'relic_symbol', 'relic_id_left2', 'relic_name', 'relic_common_name', 'pattern_usage', 'pattern_usage_detail', 'material', 'source', 'era', 'pattern_type', 'pattern_type_detail', 'relic_id_right2', '1문단', '2문단', '3문단', '4문단', '5문단', '6문단', '1paragraph', '2paragraph', '3paragraph', '4paragraph', '5paragraph', '6paragraph']
['check', 'relic_id_img', 'relic_id', 'relic_name', 'relic_name_eng', 'relic_common_name', 'relic_common_name_eng', 'pattern_usage', 'pattern_usage_id', 'pattern_usage_detail', 'material', 'source', 'era', 'pattern_type', 'pattern_type_id', 'pattern_type_detail', 'pattern_type_detail_eng', 'relic_no', 'collect', 'photo_date', 'photo_eqipment', 'color', 'object_img_resolution', 'pattern_img_resolution', '

In [36]:
outerdf = outerdf.drop(['relic_id_left2','relic_id_right2'],axis=1)
outerdf['relic_id'] = outerdf['relic_id_img']
outerdf = outerdf[master_columns]


In [37]:
print(f'master inner join rows={mas_inner_rows}, outer join rows={mas_outer_rows}, meta notfound={meta_notfound_rows}, caption notfound={cap_notfound_rows}')

master inner join rows=14163, outer join rows=27054, meta notfound=128, caption notfound=12888


In [38]:
if meta_notfound_rows:
    print(f'relic_id_img not found at meta count={meta_notfound_rows}') 
    print(f'relic_id_img not found at meta values:\n{meta_notfound_values}') 

relic_id_img not found at meta count=128
relic_id_img not found at meta values:
2744     PS0100101200100217800000_A1
3303     PS0100101410100566900000_A1
3410     PS0100101410101060900000_A1
10462                       21248_A1
10463                       21249_A1
                    ...             
26593    PS0100100800100882600000_A3
26595    PS0100100800100883000000_A2
26596    PS0100100800100883000000_A3
26598    PS0100100800100889300000_A2
26599    PS0100100800100889300000_A3
Name: relic_id_img, Length: 128, dtype: object


In [39]:
if cap_notfound_rows:
    print(f'relic_id_img not found at caption count={cap_notfound_rows}') 
    print(f'relic_id_img not found at caption values:\n{cap_notfound_values}') 

relic_id_img not found at caption count=12888
relic_id_img not found at caption values:
2        PS0100200100103812200000_A1
5        PS0100200100103812500000_A1
6        PS0100200100103812600000_A1
7        PS0100200100103812700000_A1
10       PS0100200100103971100000_A1
                    ...             
27049    PS0100308700200317400000_A2
27050    PS0100308700200317400000_A3
27051    PS0100308700200737600000_A1
27052    PS0100308700200737600000_A2
27053    PS0100308700200737600000_A3
Name: relic_id_img, Length: 12888, dtype: object


In [40]:
dataframe_info(innerdf, datadisplay=False)

shape: (14163, 38)
columns:
['check', 'relic_id_img', 'relic_id', 'relic_name', 'relic_name_eng', 'relic_common_name', 'relic_common_name_eng', 'pattern_usage', 'pattern_usage_id', 'pattern_usage_detail', 'material', 'source', 'era', 'pattern_type', 'pattern_type_id', 'pattern_type_detail', 'pattern_type_detail_eng', 'relic_no', 'collect', 'photo_date', 'photo_eqipment', 'color', 'object_img_resolution', 'pattern_img_resolution', 'object_file_name', 'relic_symbol', '1문단', '2문단', '3문단', '4문단', '5문단', '6문단', '1paragraph', '2paragraph', '3paragraph', '4paragraph', '5paragraph', '6paragraph']


In [41]:
dataframe_info(outerdf, datadisplay=False)

shape: (27054, 38)
columns:
['check', 'relic_id_img', 'relic_id', 'relic_name', 'relic_name_eng', 'relic_common_name', 'relic_common_name_eng', 'pattern_usage', 'pattern_usage_id', 'pattern_usage_detail', 'material', 'source', 'era', 'pattern_type', 'pattern_type_id', 'pattern_type_detail', 'pattern_type_detail_eng', 'relic_no', 'collect', 'photo_date', 'photo_eqipment', 'color', 'object_img_resolution', 'pattern_img_resolution', 'object_file_name', 'relic_symbol', '1문단', '2문단', '3문단', '4문단', '5문단', '6문단', '1paragraph', '2paragraph', '3paragraph', '4paragraph', '5paragraph', '6paragraph']


In [42]:
# debuging
if meta_notfound_rows:
    with open(f'./data/master_notfound_meta-{filename_suffix()}.txt', 'w') as file:
         pprint(meta_notfound_values.tolist(), stream=file)
if cap_notfound_rows:
    with open(f'./data/master_notfound_caption-{filename_suffix()}.txt', 'w') as file:
        pprint(cap_notfound_values.tolist(), stream=file)
                
innerdf.to_excel(f'./data/innerbook-{filename_suffix()}.xlsx', index=False)
outerdf.to_excel(f'./data/outerbook-{filename_suffix()}.xlsx', index=False)


In [43]:
## Write to master sheet with outerdf

In [44]:
# prepare master sheet with header only, trucate datarows 
masheet = owb1['master']

# trucate data
truncate_rows_except_headers(masheet, header_row_count=master_heads)

# fill master sheet with outerdf 
for row in outerdf.itertuples(index=False): 
    masheet.append(row)

# set text type and max width 
text_and_width(masheet, max_width=28) 


sheet=master truncate rows=27054 except header_row_count=4


In [45]:
workbook_info(owb1, datadisplay=True)


Workbook Properties:
Modified: 2024-12-13 07:29:36.679685
Number of sheets: 1
Names of sheets: ['master']

Sheet: master
  Columns: 38
  Rows: 27058
  Nonempty rows: 27058
  First 5 rows:
    (None, None, 'Master', None, None, None, None, 'O', None, None, 'o', None, None, 'o', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
    (None, None, '메타데이터', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, '직접촬영만 해당', None, None, None, None, None, None, '한글 캡션', None, None, None, None, None, '영어 캡션', None, None, None, None, None)
    ('check', 'relic_id_img', 'RELIC_ID', 'relic_name', 'relic_name_eng', 'relic_common_name', 'relic_common_name_eng', 'pattern_usage', 'pattern_usage_id', 'pattern_usage_detail', 'material', 'source', 'era', 'pattern_type', 'pattern_type_id', 'pattern_type_detail', 'pattern_type_detail_eng', 'relic_no', 'collect', 'photo_date

In [46]:
# master saving
filename = f'./data/master-{filename_suffix()}.xlsx'
owb1.save(filename) 
print(f'result file={filename}') 

result file=./data/master-14868-163302.xlsx


In [47]:
end_time = time.time()
print(f"Done, eplapsed seconds = {end_time - start_time}")

# copy file to \\BlueServer\임시폴더\이남구임시파일\

Done, eplapsed seconds = 249.73599982261658
